In [1]:
# Load libraries
import pandas as pd
from sklearn.ensemble import RandomForestRegressor # Import Decision Tree Classifier
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split # Import train_test_split function
from sklearn import metrics #Import scikit-learn metrics module for accuracy calculation
import numpy as np
import gensim

wv = gensim.models.word2vec.Word2Vec.load("updated_google_corpus")

TESTING_MODE = True

# load dataset
train_data = pd.read_csv("train.csv")
test_data = pd.read_csv("test.csv")
test_data = test_data.drop("index",axis=1)
data = pd.concat([train_data,test_data],sort=False)

In [2]:
x = pd.DataFrame()
x["s"] = np.arange(10)
x["f"] = pd.Series(["2"])

In [3]:
def rem_na(value):
    if str(value)=="nan":
            return "missing"
    return value
       

In [4]:
for col in train_data.columns:
    train_data[col] = train_data[col].apply(lambda x:rem_na(x))

In [ ]:
train_data["province"]

In [ ]:
len(train_data["title"].unique())

In [5]:
if TESTING_MODE:
    d = pd.get_dummies(train_data[["country","taster_name"]])
    d["points"]= train_data["points"]
    d["id"] = train_data['id']
else:
    d = pd.get_dummies(data[["country","region_2","province","taster_name","variety"]])
    d["points"]= data["points"]
    d["id"] = data['id']

In [6]:
if TESTING_MODE:
    for col in ["description","region_1",'designation',"region_2","province","variety",'title']:
        try:
            vecs = train_data[col].apply(lambda words:sum([wv[word.lower()] for word in words.split()])*1/len(words.split()))
        #description_vecs = train_data["description"].apply(lambda words:sum([wv[word.lower()] for word in words.split()])*1/len(words.split()))
            for i in range(300):
                d["vec_"+col+"_dim_"+str(i)]=vecs.apply(lambda v:v[i])
        except Exception as e:
            print(e)
            for i in range(300):
                d["vec_"+col+"_dim_"+str(i)]= 0
else:
    description_vecs = data["description"].apply(lambda words:sum([wv[word.lower()] for word in words.split()])*1/len(words.split()))
    for i in range(300):
        d["vec_dim_"+str(i)] = description_vecs.apply(lambda v:v[i])

/usr/lib/python3/dist-packages/ipykernel_launcher.py:4: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  after removing the cwd from sys.path.


"word 'd'aix-en-provence' not in vocabulary"
"word 'lellè' not in vocabulary"
"word 'mendocino/lake' not in vocabulary"
"word 'terras' not in vocabulary"
"word 'pinela' not in vocabulary"
"word 'county)' not in vocabulary"


In [7]:
if TESTING_MODE:
    X_train, X_test, y_train, y_test = train_test_split(d, train_data["price"], test_size=0.3, random_state=1) # 70% training and 30% test
else:
    X_train,y_train,X_test, y_test = d[:175000], data["price"][:175000], d[175000:], data["price"][175000:]

In [9]:
len(X_train.columns)

2168

In [10]:
X_train = X_train.drop(["id"],axis=1)

In [11]:
%%time
# Create Decision Tree classifer object
clf = RandomForestRegressor(50,max_depth=75,min_samples_split=2,n_jobs=10)
# Train Decision Tree Classifer
clf = clf.fit(X_train,y_train)

CPU times: user 1h 39min 44s, sys: 20.3 s, total: 1h 40min 4s
Wall time: 19min 27s


In [12]:
#Predict the response for test dataset
ids = X_test["id"]
X_test = X_test.drop(["id"],axis=1)

y_pred = clf.predict(X_test)

In [13]:
print(y_test[:10],y_pred[:10])

10709      9.0
27315     28.0
129593    38.0
23186     17.0
61957     18.0
128582    22.0
83535     20.0
65190     13.0
100308    28.0
24450     16.0
Name: price, dtype: float64 [17.1  54.94 33.56 47.92 18.14 22.54 18.76 13.72 23.42 16.82]


In [14]:
y_pred.shape

(52500,)

In [ ]:
result = pd.DataFrame()
result['id'] = ids
result["price"] = y_pred

from math import ceil
result["price"]=result["price"].apply(ceil)

result.to_csv('out_rfr_.csv',index=False)

In [15]:
print("Accuracy:",metrics.accuracy_score(y_test, np.int64(y_pred)))


Accuracy: 0.07432380952380953


In [16]:
print('RMSE:', np.sqrt(metrics.mean_squared_error(y_test, y_pred)))

RMSE: 24.024303077231327


In [ ]:
20,-  : 23.649820840960377
50,-  : 22.99395012474037  
100,- : 22.884203101828
20,20 : 24.206928409441964
20,50 : 23.654831350249047
20,100 : 24.032463084435644
20,75 : 23.79979766920799
100,75 : 22.846035679936353 -> 33.87389
10,20,10: 26.076570435580027
10,20,2: 25.116716569846435
10,20,20: 26.74973180661442

In [ ]:
y_pred[np.argmin(y_pred)]